# Assignment 3

Stuart

a = 2353
b = 20837
c = 500551
d = 29213

Q6 = iii)

In [1]:
a = 2353
b = 20837
c = 500551
d = 29213

__Question 1 (10 marks):__

For the number $a$ given to you on page 1 answer the following:

* Show that a can be written as the sum of squares in two different ways. 
    
* Hence apply Euler’s method to factorise a.

In [ ]:
# Write a function for Euler's method

__Question 2 (10 marks):__

Take the number $b$ assigned to you on page 1 and apply the gcd method to find its smallest factor.
    
What is the value of $P_0$ that you need to guarantee finding a factor given that $b$ is composite?

__Question 3 (10 marks):__

Take the number $c$ assigned to you on page and use the $p-1$ method to find one factor of $c$. You may assume that $c-1$ factorises into primes of size $< 100$.

__Question 4 (10 marks)__

Take the number $d$ assigned to you on page and use the $p-1$ method to find both factors of $d$. You may NOT use the maple procedure provided in weblearn.

__Question 5 (15 marks)__

Take the same number $d$ and now factorise using the Quadratic Sieve method. You may use Maple commands included in the week 10 workshop folder.

__Question 6 (15 marks)__

The Maple worksheet in Weblearn for this assignment shows part of an attempt to factorise $N = 9263 = 59*157$ using the Number Field Sieve. In this we claim the following:

&nbsp;&nbsp;&nbsp;&nbsp;$A = [0 , 1, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 2
    
&nbsp;&nbsp;&nbsp;&nbsp;$B = [-1 ,- 1, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 3
    
&nbsp;&nbsp;&nbsp;&nbsp;$C = [1, 0 , 1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 5  
    
&nbsp;&nbsp;&nbsp;&nbsp;$D = [1 , 1, -1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 11
    
&nbsp;&nbsp;&nbsp;&nbsp;$E = [1 , -2, 0]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$   with norm = 17
    
&nbsp;&nbsp;&nbsp;&nbsp;$F = [3 , 0, -1]$ is a prime(irreducible) element of $\mathbb{Z}(\sqrt[3]{-2})$  with norm = 23

We also derive a $48 x 15$ matrix $R$ consisting of values of $a$ and $b$ such that $a + 21b$ can be factorised using small primes and $[a, b, 0]$ can 
be factorised in Z(∛(-2) using just the primes ${A, B, C, D, E, F}$ and a unit element U = $[1, 1, 0]$.

&nbsp;&nbsp;&nbsp;&nbsp;Prove the statement *iii)* $C$ above allocated to you on page 1 using the definition of a norm.
    
&nbsp;&nbsp;&nbsp;&nbsp;Show that rows 23, 37, 41 and 45 of the matrix  form a linearly dependent set modulo 2
    
&nbsp;&nbsp;&nbsp;&nbsp;Hence find an equation of the form $u_2 – v_2$ such that $u_2 – v_2$ and $N$ have a common factor of $59$ thus 
factorising N. You may use the Maple procedure for multiplication in $\mathbb{Z}(\sqrt[3]{-2})$ to help you find $u$ and $v$.

__Q7__

Compare the methods for integer factorisation you have seen in the module and summarises their strengths and weaknesses, including the size of numbers that can be factorised, usability and whether or not they work for a broad range of numbers.  